In [2]:
!git clone https://github.com/viai957/Optimal-Portfolio-Transactions.git

Cloning into 'Optimal-Portfolio-Transactions'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 35 (delta 9), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (35/35), 763.47 KiB | 7.56 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
%cd Optimal-Portfolio-Transactions
from ddpg_agent import Agent

/content/Optimal-Portfolio-Transactions


In [8]:
import numpy as np
import torch
from collections import deque #datastructure that lets you add and remove item from both ends
# Create simulation environment
import syntheticChrissAlmgren as sca
env = sca.MarketEnvironment() #initialize the market environment

# Initialize DDPG agent with actor and critic networks
agent = Agent(state_size=env.observation_space_dimension(), action_size=env.action_space_dimension(), random_seed=0)
# agent learns to make trading decisions

# Set parameters for the simulation
lqt = 60  # Liquidation time
n_trades = 60  # Number of trades
tr = 1e-6  # Trader's risk aversion
episodes = 3000 # Number of episodes to train

shortfall_hist = np.array([]) #record the final shortfall for all episodes
shortfall_deque = deque(maxlen=100) #record the last 100 episodes' shortfall

for episode in range(episodes): #loop through the training episodes
    cur_state = env.reset(seed=episode, liquid_time=lqt, num_trades=n_trades, lamb=tr) #reset the environment for the new episode
    env.start_transactions() #start the simulation

    for i in range(n_trades + 1): #loop through the trade within an episode
        # Action Prediction: The agent predicts the optimal trade amount based on the current state
        action = agent.act(cur_state, add_noise=True) #amt of stock to trade

        # Execytion and feedback - Perform the action and receive the new state, reward(minimize the implementation shortfall), and additional information
        new_state, reward, done, info = env.step(action) #execute the trade

        # Learning - The agent updates its networks (actor and critic) using the experiences collected in the replay buffer
        agent.step(cur_state, action, reward, new_state, done) #agent learns from the outcome of the trade

        #Shortfall Calculation - The implementation shortfall is calculated at the end of each episode, and the goal is to minimize this metric across episodes

        # Move to the next state
        cur_state = new_state #update the current state

        if info.done: #to check if the episode is finished
            shortfall_hist = np.append(shortfall_hist, info.implementation_shortfall) #recod the final shortfall for one epi
            shortfall_deque.append(info.implementation_shortfall) #add the shortfall to recent history
            break #move to the next epi

    if (episode + 1) % 100 == 0:  # Print average shortfall over last 100 episodes
        print(f'Episode [{episode + 1}/{episodes}]\tAverage Shortfall: ${np.mean(shortfall_deque):,.2f}')

print(f'\nAverage Implementation Shortfall: ${np.mean(shortfall_hist):,.2f} \n')
#print overall average shortfall

Episode [100/3000]	Average Shortfall: $2,276,855.50
Episode [200/3000]	Average Shortfall: $2,562,256.53
Episode [300/3000]	Average Shortfall: $2,562,500.00
Episode [400/3000]	Average Shortfall: $2,562,500.00
Episode [500/3000]	Average Shortfall: $2,562,500.00
Episode [600/3000]	Average Shortfall: $2,562,500.00
Episode [700/3000]	Average Shortfall: $2,562,500.00
Episode [800/3000]	Average Shortfall: $2,562,500.00
Episode [900/3000]	Average Shortfall: $2,562,500.00
Episode [1000/3000]	Average Shortfall: $2,562,500.00
Episode [1100/3000]	Average Shortfall: $2,562,500.00
Episode [1200/3000]	Average Shortfall: $2,562,500.00
Episode [1300/3000]	Average Shortfall: $2,562,500.00
Episode [1400/3000]	Average Shortfall: $2,562,500.00
Episode [1500/3000]	Average Shortfall: $2,562,500.00
Episode [1600/3000]	Average Shortfall: $2,562,500.00
Episode [1700/3000]	Average Shortfall: $2,562,500.00
Episode [1800/3000]	Average Shortfall: $2,562,500.00
Episode [1900/3000]	Average Shortfall: $2,562,500.00
Ep